# kutubxonalarni chaqiramiz

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
labeller = LabelEncoder()

from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method='yeo-johnson', standardize=True)

# kesib olingan 25 ming qator datani chaqiramiz

In [104]:
df = pd.read_csv(r'C:\Users\bunyo\OneDrive\Desktop\git_project\data\raw_data\sliced_data\sliced_data.csv')

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           25000 non-null  object 
 1   Location       25000 non-null  object 
 2   MinTemp        24754 non-null  float64
 3   MaxTemp        24821 non-null  float64
 4   Rainfall       24438 non-null  float64
 5   Evaporation    14243 non-null  float64
 6   Sunshine       13014 non-null  float64
 7   WindGustDir    23222 non-null  object 
 8   WindGustSpeed  23235 non-null  float64
 9   WindDir9am     23186 non-null  object 
 10  WindDir3pm     24277 non-null  object 
 11  WindSpeed9am   24684 non-null  float64
 12  WindSpeed3pm   24490 non-null  float64
 13  Humidity9am    24540 non-null  float64
 14  Humidity3pm    24245 non-null  float64
 15  Pressure9am    22422 non-null  float64
 16  Pressure3pm    22442 non-null  float64
 17  Cloud9am       15392 non-null  float64
 18  Cloud3

# class orqli dataga ishlov beramiz

In [105]:
class Preprocessing:
    def __init__(self,df):
        self.df = df
        
# Encoding qiluvchi
    def encoding_qilish(self):
        for col in self.df.columns:
            if self.df[col].dtype == 'object':
                if self.df[col].nunique() <= 3:
                    new_df = pd.get_dummies(self.df[col], prefix='New', dtype=int)
                    self.df.drop(columns=[col],inplace=True)
                    self.df = pd.concat([self.df,new_df],axis=1)
                else:
                    self.df[col] = labeller.fit_transform(self.df[col])
        return self
    

# Scale qiluvchi
    def scaling_qilish(self): 
        for cols in self.df.columns: 
            if self.df[cols].dtype != 'object' and self.df[cols].name != 'RainTomorrow': 
                self.df[cols] = minmax.fit_transform(self.df[[cols]]) 
        return self


# To'ldruvchi
    def fillingNan(self):
        for cols in self.df.columns:
            if self.df[cols].isnull().any():
                if self.df[cols].dtype == 'object':
                    self.df[cols].fillna(self.df[cols].mode()[0], inplace=True)
                else:
                    self.df[cols].fillna(self.df[cols].mean(),inplace=True)
        return self         
    

# skewnewssni tog'irlash (log bilan)
    def log_transform(self, chegara = 0.7):
        skewness = self.df.skew()
        features_log = skewness[(skewness >= chegara)].index.tolist()
        
        for col in features_log:
            if (self.df[col] > 0).all():  # only positive values
                self.df[col] = np.log1p(self.df[col])
        return self
    

# skewnessni power bilan togirlash
    def power_transform(self, chegara = 0):
        skewness = self.df.skew()
        Pow_transform = skewness[(skewness < chegara)].index.to_list()

        for col in Pow_transform:
            self.df[col] = pt.fit_transform(self.df[[col]])  #pt atamasi power Transform degani tepada shunday sozga biriktirilgan
        return self.df

# Target qiymatni numeric ga o'tkizamiz

In [107]:
df['RainTomorrow'].replace({'No': 0, 'Yes': 1},inplace = True)

# class ni ishlatib dataga ishlov beramiz

In [109]:
pre_processing = Preprocessing(df)
df1 = pre_processing.fillingNan().encoding_qilish().scaling_qilish().log_transform().power_transform()

In [111]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           25000 non-null  float64
 1   Location       25000 non-null  float64
 2   MinTemp        25000 non-null  float64
 3   MaxTemp        25000 non-null  float64
 4   Rainfall       25000 non-null  float64
 5   Evaporation    25000 non-null  float64
 6   Sunshine       25000 non-null  float64
 7   WindGustDir    25000 non-null  float64
 8   WindGustSpeed  25000 non-null  float64
 9   WindDir9am     25000 non-null  float64
 10  WindDir3pm     25000 non-null  float64
 11  WindSpeed9am   25000 non-null  float64
 12  WindSpeed3pm   25000 non-null  float64
 13  Humidity9am    25000 non-null  float64
 14  Humidity3pm    25000 non-null  float64
 15  Pressure9am    25000 non-null  float64
 16  Pressure3pm    25000 non-null  float64
 17  Cloud9am       25000 non-null  float64
 18  Cloud3

# preprocessed datani saqlaymiz

In [ ]:
import os 

In [115]:
folder = r'C:\Users\bunyo\OneDrive\Desktop\git_project\data\preprocessed_data'
path = os.path.join(folder,'preprocessed_data.csv')
df1.to_csv(path, index=False)